In [ ]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "63874452-8869-4663-b123-dfb83abef167",
   "metadata": {},
   "source": [
    "# 1. 라이브러리"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 1,
   "id": "6c25f29d-4344-4ba9-8ee6-c24c8b77c945",
   "metadata": {},
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "\n",
    "import os\n",
    "import glob\n",
    "import time\n",
    "\n",
    "from datetime import datetime\n",
    "\n",
    "from urllib.request import urlopen\n",
    "from urllib.parse import urlencode, unquote, quote_plus\n",
    "from xml.etree.ElementTree import parse\n",
    "import json\n",
    "import urllib\n",
    "import requests\n",
    "import xmltodict\n",
    "\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "04428ee0-cfac-48d8-b730-ec158945535a",
   "metadata": {},
   "source": [
    "# 2. API 기본 형식"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 2,
   "id": "5b0a1800-e9b5-4e69-8072-92a9ec89a109",
   "metadata": {},
   "outputs": [],
   "source": [
    "# API url\n",
    "url = 'url 주소'\n",
    "\n",
    "queryParams = '?' + urlencode({\n",
    "\n",
    "                # 서비스키\n",
    "                quote_plus('ServiceKey') : '서비스키',\n",
    "                quote_plus('pageNo') : 'page',           # 페이지 설정\n",
    "                quote_plus('numOfRows') : 'row',         # 페이지 당 조회 행 개수 설정\n",
    "                quote_plus('dataType') : 'XML',          # datatype = xml\n",
    "                quote_plus('dataCd') : 'ASOS',           # ASOS 관측 기구 조회\n",
    "                quote_plus('dateCd') : 'day or hr',      # 시간 조회\n",
    "                quote_plus('startDt') : 'start day',     # 시작 일자 설정\n",
    "                quote_plus('endDt') : 'end day',         # 끝 일자 설정\n",
    "                quote_plus('startHh') : 'start time',    # 시작 시간 설정\n",
    "                quote_plus('endHh') : 'end time',        # 끝 시간 설정\n",
    "                quote_plus('stnIds') : 'loc'})           # 지점번호 선택\n",
    "\n",
    "# request 객체\n",
    "request = urllib.request.Request(url + unquote(queryParams))\n",
    "\n",
    "# url 응답 객체 저장\n",
    "response_body = urlopen(request, timeout=60).read()\n",
    "\n",
    "# 디코딩\n",
    "decode_data = response_body.decode('utf-8')\n",
    "\n",
    "# string인 xml 파싱\n",
    "xml_parse = xmltodict.parse(decode_data)     \n",
    "xml_dict = json.loads(json.dumps(xml_parse))"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "d3643cda-cff0-4e4e-a13c-4435fe5fc122",
   "metadata": {},
   "source": [
    "지상 기상 관측의 대표적인 방법은 종관기상관측(ASOS)와 방재기상관측(AWS)가 있습니다.\n",
    "\n",
    "여러 지역에서 같은 시각에 관측하는 종관규모의 관측을 주로 기상데이터로 사용하기 때문에 본 프로젝트에서 종관기상관측(ASOS)를 이용합니다."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "b167ea6f-793a-4b80-b168-764e6e4ca260",
   "metadata": {},
   "source": [
    "# 3. 지점 번호 선언"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 3,
   "id": "8ecd6b8a-2a72-41d1-8fff-5d6c27874392",
   "metadata": {},
   "outputs": [],
   "source": [
    "# API부를 때 필요한 지점 번호 리스트\n",
    "std_list = [90, 93, 95, 98, 99, 100, 101, 102, 104, 105, 106, 108, 112, 114, 115, 119, 121, 127, 129, 130,\n",
    "            131, 133, 135, 136, 137, 138, 140, 143, 146, 152, 155, 156, 159, 162, 165, 168, 169, 170, 172,\n",
    "            174, 177, 184, 185, 188, 189, 192, 201, 202, 203, 211, 212, 216, 217, 221, 226, 232, 235, 236,\n",
    "            238, 239, 243, 244, 245, 247, 248, 251, 252, 253, 254, 255, 257, 258, 259, 260, 261, 262, 263,\n",
    "            264, 266, 268, 271, 272, 273, 276, 277, 278, 279, 281, 283, 284, 285, 288, 289, 294, 295]"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "80bc0320-5bca-4a3d-8c4d-6d1b0789cfa4",
   "metadata": {},
   "source": [
    "# 4-1. 2011 ~ 2020년 일별 기후 데이터"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 4,
   "id": "edb0b654-2e3f-48f6-a85c-d2e1abeb188c",
   "metadata": {
    "jupyter": {
     "outputs_hidden": true
    },
    "scrolled": true,
    "tags": []
   },
   "outputs": [],
   "source": [
    "# API url\n",
    "url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'\n",
    "\n",
    "# 데이터 프레임 컬럼 설정\n",
    "col_list = ['지점_번호', '지점명', '시간', '평균_기온', '최저_기온', '최저_기온_시각', '최고_기온', '최고_기온_시각',\n",
    "            '강수_계속시간', '10분_최다강수량', '10분_최다강수량_시각', '1시간_최다강수량', '1시간_최다_강수량_시각',\n",
    "            '일강수량', '최대_순간풍속', '최대_순간_풍속_풍향', '최대_순간풍속_시각', '최대_풍속', '최대_풍속_풍향',\n",
    "            '최대_풍속_시각', '평균_풍속', '풍정합', '최다_풍향', '평균_이슬점온도', '최소_상대습도', '평균_상대습도_시각',\n",
    "            '평균_상대습도', '평균_증기압', '평균_현지기압', '최고_해면_기압', '최고_해면기압_시각', '최저_해면기압',\n",
    "            '최저_해면기압_시각', '평균_해면기압', '가조시간', '합계_일조_시간', '1시간_최다_일사_시각', '1시간_최다_일사량',\n",
    "            '합계_일사량', '일_최심신적설', '일_최심신적설_시각', '일_최심적설', '일_최심적설_시각', '합계_3시간_신적설',\n",
    "            '평균_전운량', '평균_중하층운량', '평균_지면온도', '최저_초상온도', '평균_5cm_지중온도', '평균10cm_지중온도',\n",
    "            '평균_20cm_지중온도', '평균_30cm_지중온도', '0.5m_지중온도', '1.0m_지중온도', '1.5m_지중온도', '3.0m_지중온도',\n",
    "            '5.0m_지중온도', '합계_대형증발량', '합계_소형증발량', '9-9강수', '일기현상', '안개_계속_시간']\n",
    "\n",
    "breaker = False\n",
    "for std in std_list:\n",
    "    df = pd.DataFrame()\n",
    "    print(f'▷ 지점번호 {std} 시작 -- {datetime.now().time()}')\n",
    "    \n",
    "    if breaker == True:\n",
    "        pass\n",
    "\n",
    "    page = 1\n",
    "    cnt = 0\n",
    "    breaker = False\n",
    "\n",
    "    # 네트워크 오류 발생으로 인한 누락 방지를 위해 try-except문으로 무한 루프로 구현\n",
    "    while True:\n",
    "        try:\n",
    "            queryParams = '?' + urlencode({\n",
    "\n",
    "                    # 서비스 키\n",
    "                    quote_plus('serviceKey') : 'LlGn22WX8%2BUx36D9E%2FNkES4XSfhz7UTelVHghAms7Oxdee3fjCtplPkUpczGO%2Bc2BU%2BV%2Fh5Mh%2BWEEOk2h4yetQ%3D%3D',\n",
    "                    quote_plus('pageNo') : f'{page}',      # 페이지 설정\n",
    "                    quote_plus('numOfRows') : '366',       # 페이지 당 조회 행 개수 설정\n",
    "                    quote_plus('dataType') : 'XML',        # datatype = xml\n",
    "                    quote_plus('dataCd') : 'ASOS',         # ASOS 관측 기구 조회\n",
    "                    quote_plus('dateCd') : 'DAY',          # 일자 조회\n",
    "                    quote_plus('startDt') : f'20110101',   # 시작 일자 설정\n",
    "                    quote_plus('endDt') : f'20201231',     # 끝 일자 설정\n",
    "                    quote_plus('stnIds') : f'{std}'})      # 지점번호 선택\n",
    "\n",
    "            # request 객체\n",
    "            request = urllib.request.Request(url + unquote(queryParams))\n",
    "\n",
    "            # url 응답 객체 저장\n",
    "            response_body = urlopen(request, timeout=60).read()\n",
    "\n",
    "            # 디코딩\n",
    "            decode_data = response_body.decode('utf-8')\n",
    "\n",
    "            # string인 xml 파싱\n",
    "            xml_parse = xmltodict.parse(decode_data)   \n",
    "            xml_dict = json.loads(json.dumps(xml_parse))\n",
    "            result = xml_dict['response']['header']['resultMsg']\n",
    "\n",
    "            # 무한 루프를 돌리기 때문에 try-except를 빠져나오지 못하는 'NO_DATA'오류일 경우 반복문을 빠져나오게 함\n",
    "            if result == 'NO_DATA':\n",
    "                break\n",
    "            dicts = xml_dict['response']['body']['items']['item']\n",
    "\n",
    "            df_temp = pd.DataFrame(columns = col_list)\n",
    "\n",
    "            # 받은 데이터 데이터 프레임에 적재\n",
    "            for j in range(len(dicts)):\n",
    "                df_temp.loc[j] = dicts[j].values()\n",
    "\n",
    "            df = pd.concat([df, df_temp])\n",
    "            print(f'{page}page 완료', end=', ')\n",
    "            page += 1\n",
    "            cnt += 1\n",
    "            if cnt == 12:\n",
    "                breaker = True\n",
    "                break\n",
    "\n",
    "        except:\n",
    "            print(f'{page}page 오류 다시 시작', end=', ')\n",
    "            cnt += 1\n",
    "            if cnt == 12:\n",
    "                break\n",
    "\n",
    "        # 한 페이지에 366개 row의 데이터가 들어가기 때문에 10페이지가 넘어가면 무한 루프 break\n",
    "        if page == 11:\n",
    "            break\n",
    "\n",
    "    # 데이터 저장\n",
    "    if len(df.values) != 0:\n",
    "        df.to_csv(f\"./dayday/{df['지점명'].unique()[0]}\" + \"_day_fianl.csv\", index=False)\n",
    "        print()\n",
    "        print(f'지점번호 {std} 완료 -- {datetime.now().time()}\\n')\n",
    "    else:\n",
    "        print(f'지점번호 {std} NO DATA\\n')"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "d2a2a29f",
   "metadata": {},
   "source": [
    "훈련(train) 데이터로 사용할 2011 ~ 2020년 일별 데이터를 수집합니다."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "26274981-ff50-4ae2-8914-feccbfa732d2",
   "metadata": {},
   "source": [
    "# 4-2. 2021 ~ 2022년 일별 기후 데이터"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 5,
   "id": "95551577-619c-41a4-b593-946585988ce8",
   "metadata": {
    "jupyter": {
     "outputs_hidden": true
    },
    "scrolled": true,
    "tags": []
   },
   "outputs": [],
   "source": [
    "# API url\n",
    "url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'\n",
    "\n",
    "# 데이터 프레임 컬럼 설정\n",
    "col_list = ['지점_번호', '지점명', '시간', '평균_기온', '최저_기온', '최저_기온_시각', '최고_기온', '최고_기온_시각',\n",
    "            '강수_계속시간', '10분_최다강수량', '10분_최다강수량_시각', '1시간_최다강수량', '1시간_최다_강수량_시각',\n",
    "            '일강수량', '최대_순간풍속', '최대_순간_풍속_풍향', '최대_순간풍속_시각', '최대_풍속', '최대_풍속_풍향',\n",
    "            '최대_풍속_시각', '평균_풍속', '풍정합', '최다_풍향', '평균_이슬점온도', '최소_상대습도', '평균_상대습도_시각',\n",
    "            '평균_상대습도', '평균_증기압', '평균_현지기압', '최고_해면_기압', '최고_해면기압_시각', '최저_해면기압',\n",
    "            '최저_해면기압_시각', '평균_해면기압', '가조시간', '합계_일조_시간', '1시간_최다_일사_시각', '1시간_최다_일사량',\n",
    "            '합계_일사량', '일_최심신적설', '일_최심신적설_시각', '일_최심적설', '일_최심적설_시각', '합계_3시간_신적설',\n",
    "            '평균_전운량', '평균_중하층운량', '평균_지면온도', '최저_초상온도', '평균_5cm_지중온도', '평균10cm_지중온도',\n",
    "            '평균_20cm_지중온도', '평균_30cm_지중온도', '0.5m_지중온도', '1.0m_지중온도', '1.5m_지중온도', '3.0m_지중온도',\n",
    "            '5.0m_지중온도', '합계_대형증발량', '합계_소형증발량', '9-9강수', '일기현상', '안개_계속_시간']\n",
    "\n",
    "breaker = False\n",
    "for std in std_list:\n",
    "    df = pd.DataFrame()\n",
    "    print(f'▷ 지점번호 {std} 시작 -- {datetime.now().time()}')\n",
    "    \n",
    "    if breaker == True:\n",
    "        pass\n",
    "    page = 1\n",
    "    cnt = 0\n",
    "    breaker = False\n",
    "\n",
    "    # 네트워크 오류 발생으로 인한 누락 방지를 위해 try-except문으로 무한 루프로 구현\n",
    "    while True:\n",
    "        try:\n",
    "            queryParams = '?' + urlencode({\n",
    "\n",
    "                    # 서비스 키\n",
    "                    quote_plus('serviceKey') : 'LlGn22WX8%2BUx36D9E%2FNkES4XSfhz7UTelVHghAms7Oxdee3fjCtplPkUpczGO%2Bc2BU%2BV%2Fh5Mh%2BWEEOk2h4yetQ%3D%3D',\n",
    "                    quote_plus('pageNo') : f'{page}',     # 페이지 설정\n",
    "                    quote_plus('numOfRows') : '366',      # 페이지 당 조회 행 개수 설정\n",
    "                    quote_plus('dataType') : 'XML',       # datatype = xml\n",
    "                    quote_plus('dataCd') : 'ASOS',        # ASOS 관측 기구 조회\n",
    "                    quote_plus('dateCd') : 'DAY',         # 일자 조회\n",
    "                    quote_plus('startDt') : f'20210101',  # 시작 일자 설정\n",
    "                    quote_plus('endDt') : f'20221231',    # 끝 일자 설정\n",
    "                    quote_plus('stnIds') : f'{std}'})     # 지점번호 선택\n",
    "\n",
    "            # request 객체\n",
    "            request = urllib.request.Request(url + unquote(queryParams))\n",
    "\n",
    "            # url 응답 객체 저장\n",
    "            response_body = urlopen(request, timeout=60).read()\n",
    "\n",
    "            # 디코딩\n",
    "            decode_data = response_body.decode('utf-8')\n",
    "\n",
    "            # string인 xml 파싱\n",
    "            xml_parse = xmltodict.parse(decode_data)    \n",
    "            xml_dict = json.loads(json.dumps(xml_parse))\n",
    "            result = xml_dict['response']['header']['resultMsg']\n",
    "\n",
    "            # 무한 루프를 돌리기 때문에 try-except를 빠져나오지 못하는 'NO_DATA'오류일 경우 반복문을 빠져나오게 함\n",
    "            if result == 'NO_DATA':\n",
    "                # breaker = True\n",
    "                break\n",
    "            dicts = xml_dict['response']['body']['items']['item']\n",
    "\n",
    "            df_temp = pd.DataFrame(columns = col_list)\n",
    "\n",
    "            # 받은 데이터 데이터 프레임에 적재\n",
    "            for j in range(len(dicts)):\n",
    "                df_temp.loc[j] = dicts[j].values()\n",
    "\n",
    "            df = pd.concat([df, df_temp])\n",
    "            print(f'{page}page 완료', end=', ')\n",
    "            page += 1\n",
    "            cnt += 1\n",
    "            if cnt == 6:\n",
    "                breaker = True\n",
    "                break\n",
    "\n",
    "        except:\n",
    "            print(f'{page}page 오류 다시 시작', end=', ')\n",
    "            cnt += 1\n",
    "            if cnt == 6:\n",
    "                break\n",
    "\n",
    "        # 한 페이지에 366개 row의 데이터가 들어가기 때문에 2페이지가 넘어가면 무한 루프 break\n",
    "        if page == 3:\n",
    "            break\n",
    "\n",
    "    # 데이터 저장\n",
    "    if len(df.values) != 0:\n",
    "        df.to_csv(f\"./data/data_day/test/{df['지점명'].unique()[0]}\" + \"_day_fianl.csv\", index=False)\n",
    "        print()\n",
    "        print(f'지점번호 {std} 완료 -- {datetime.now().time()}\\n')\n",
    "    else:\n",
    "        print(f'지점번호 {std} NO DATA\\n')"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "0224ef51",
   "metadata": {},
   "source": [
    "테스트(test) 데이터로 사용할 2021 ~ 2022년 일별 데이터를 수집합니다."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "e651efac-c85a-4cad-8bcf-4969990fd918",
   "metadata": {},
   "source": [
    "# 4-3. 2011 ~ 2020년 시간별 기후 데이터"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 6,
   "id": "8c8f0f34-8159-43b2-9162-65d6639b315d",
   "metadata": {},
   "outputs": [],
   "source": [
    "# API를 부르는 데에 걸리는 시간을 파악하기 위해 선언\n",
    "from datetime import datetime\n",
    "\n",
    "# API url\n",
    "url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'\n",
    "\n",
    "for std in std_list:\n",
    "    df = pd.DataFrame()\n",
    "    print(f'▷ 지점번호 {std} 시작 -- {datetime.now().time()}')\n",
    "    breaker = False\n",
    "    for year in range(11, 21, 1):\n",
    "        print(f'{year}년 시작', end = ', ')\n",
    "        if breaker == True:\n",
    "            break\n",
    "        page = 1\n",
    "\n",
    "        # 네트워크 오류 발생으로 인한 누락 방지를 위해 try-exept문으로 무한 루프로 구현\n",
    "        while True:\n",
    "            try:\n",
    "                queryParams = '?' + urlencode({\n",
    "\n",
    "                        # 서비스 키\n",
    "                        quote_plus('ServiceKey') : '%2FNJpEb%2B4mPrdl%2BjK0N5XfHVW2zDqJiBAj7IwHykCI4aJExuDhIr9VfQeUYZKCAeIDNMRk%2BfUF0x9J47VhrxPyQ%3D%3D',\n",
    "                        quote_plus('pageNo') : f'{page}',         # 페이지 설정\n",
    "                        quote_plus('numOfRows') : '880',          # 페이지 당 조회 행 개수 설정\n",
    "                        quote_plus('dataType') : 'XML',           # datatype = xml\n",
    "                        quote_plus('dataCd') : 'ASOS',            # ASOS 관측 기구 조회\n",
    "                        quote_plus('dateCd') : 'HR',              # 시간 조회\n",
    "                        quote_plus('startDt') : f'20{year}0101',  # 시작 일자 설정\n",
    "                        quote_plus('endDt') : f'20{year}1231',    # 끝 일자 설정\n",
    "                        quote_plus('startHh') : '00',             # 시작 시간 설정\n",
    "                        quote_plus('endHh') : '23',               # 끝 시간 설정\n",
    "                        quote_plus('stnIds') : f'{std}'})         # 지점번호 선택\n",
    "\n",
    "                # request 객체\n",
    "                request = urllib.request.Request(url + unquote(queryParams))\n",
    "\n",
    "                # url 응답 객체 저장\n",
    "                response_body = urlopen(request, timeout=60).read()\n",
    "\n",
    "                # 디코딩\n",
    "                decode_data = response_body.decode('utf-8')\n",
    "\n",
    "                # string인 xml 파싱\n",
    "                xml_parse = xmltodict.parse(decode_data)     \n",
    "                xml_dict = json.loads(json.dumps(xml_parse))\n",
    "                result = xml_dict['response']['header']['resultMsg']\n",
    "\n",
    "                # 무한 루프를 돌리기 때문에 try-exept 를 빠져나오지 못하는 'NO_DATA'오류일 경우 반복문을 빠져나오게 함\n",
    "                if result == 'NO_DATA':\n",
    "                    breaker = True\n",
    "                    break\n",
    "                dicts = xml_dict['response']['body']['items']['item']\n",
    "                \n",
    "                # 데이터 프레임 컬럼 설정\n",
    "                df_temp = pd.DataFrame(columns = ['날짜', '시간', '지점_번호', '지점명', '기온', '강수량', '풍속', '풍향',\n",
    "                                                  '습도','증기압', '이슬점온도', '현지기압', '해면기압', '일조', '일사', '적설',\n",
    "                                                  '3시간_신절설', '전운량', '중하층운량', '운형', '최저운고', '시정', '지면온도',\n",
    "                                                  '5cm_지중온도', '10cm_지중온도', '20cm_지중온도', '30cm_지중온도'])\n",
    "                \n",
    "                # 받은 데이터 데이터 프레임에 적재\n",
    "                for i in range(len(dicts)):\n",
    "                    df_temp.loc[i] = [dicts[i]['tm'][:10], dicts[i]['tm'][11:], dicts[i]['stnId'], dicts[i]['stnNm'],\n",
    "                                      dicts[i]['ta'], dicts[i]['rn'], dicts[i]['ws'], dicts[i]['wd'], dicts[i]['hm'],\n",
    "                                      dicts[i]['pv'], dicts[i]['td'], dicts[i]['pa'], dicts[i]['ps'], dicts[i]['ss'],\n",
    "                                      dicts[i]['icsr'], dicts[i]['dsnw'], dicts[i]['hr3Fhsc'], dicts[i]['dc10Tca'],\n",
    "                                      dicts[i]['dc10LmcsCa'], dicts[i]['clfmAbbrCd'], dicts[i]['lcsCh'], dicts[i]['vs'],\n",
    "                                      dicts[i]['ts'], dicts[i]['m005Te'], dicts[i]['m01Te'], dicts[i]['m02Te'], dicts[i]['m03Te']]\n",
    "                df = pd.concat([df, df_temp])\n",
    "                print(f'{page}page 완료', end = ', ')\n",
    "                page += 1\n",
    "\n",
    "            except:\n",
    "                print(f'{page}page 오류 다시 시작', end = ', ')\n",
    "            if page == 11:\n",
    "                break\n",
    "        print()\n",
    "        print(f'{year}년 완료 -- {datetime.now().time()}')\n",
    "    \n",
    "    # 지점 별로 csv 파일로 저장\n",
    "    if len(df.values) != 0:\n",
    "        df.to_csv(f\"./data/data_hour/test/{df['지점명'].unique()[0]}\" + \"_tm_final.csv\", index=False)\n",
    "        print(f'지점번호 {std} 완료 -- {datetime.now().time()}\\n')\n",
    "    else:\n",
    "        print(f'지점번호 {std} NO DATA\\n')"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "ec3dc9ae-4f84-4c15-973d-2de59f7653ed",
   "metadata": {},
   "source": [
    "시간별 데이터는 전처리 과정에서 일별 기후 데이터의 결측치를 대체하는 데 사용됩니다.\n",
    "\n",
    "일별 학습(train) 데이터의 결측치를 대체하기 위해서 2011 ~ 2020년의 시간별 데이터를 수집합니다.\n",
    "\n",
    "시간별 기후 데이터 수집 지표는 일별 데이터의 지표와 같습니다."
   ]
  },
  {
   "cell_type": "markdown",
   "id": "1a731f44-66c3-4e97-b7ed-9800e8a1f0f0",
   "metadata": {},
   "source": [
    "# 4-4. 2021 ~ 2022년 시간별 기후 데이터"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 7,
   "id": "178ddcfc-6cd6-4976-914e-ad78e8b7d2eb",
   "metadata": {
    "jupyter": {
     "outputs_hidden": true
    },
    "scrolled": true,
    "tags": []
   },
   "outputs": [],
   "source": [
    "# API를 부르는 데에 걸리는 시간을 파악하기 위해 선언\n",
    "from datetime import datetime\n",
    "\n",
    "# API url\n",
    "url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'\n",
    "\n",
    "for std in std_list:\n",
    "    df = pd.DataFrame()\n",
    "    print(f'▷ 지점번호 {std} 시작 -- {datetime.now().time()}')\n",
    "    breaker = False\n",
    "    for year in range(21, 23, 1):\n",
    "        print(f'{year}년 시작', end = ', ')\n",
    "        if breaker == True:\n",
    "            break\n",
    "        page = 1\n",
    "\n",
    "        # 네트워크 오류 발생으로 인한 누락 방지를 위해 try-exept문으로 무한 루프로 구현\n",
    "        while True:\n",
    "            try:\n",
    "                queryParams = '?' + urlencode({\n",
    "\n",
    "                        # 서비스 키\n",
    "                        quote_plus('ServiceKey') : '%2FNJpEb%2B4mPrdl%2BjK0N5XfHVW2zDqJiBAj7IwHykCI4aJExuDhIr9VfQeUYZKCAeIDNMRk%2BfUF0x9J47VhrxPyQ%3D%3D',\n",
    "                        quote_plus('pageNo') : f'{page}',         # 페이지 설정\n",
    "                        quote_plus('numOfRows') : '880',          # 페이지 당 조회 행 개수 설정\n",
    "                        quote_plus('dataType') : 'XML',           # datatype = xml\n",
    "                        quote_plus('dataCd') : 'ASOS',            # ASOS 관측 기구 조회\n",
    "                        quote_plus('dateCd') : 'HR',              # 시간 조회\n",
    "                        quote_plus('startDt') : f'20{year}0101',  # 시작 일자 설정\n",
    "                        quote_plus('endDt') : f'20{year}1231',    # 끝 일자 설정\n",
    "                        quote_plus('startHh') : '00',             # 시작 시간 설정\n",
    "                        quote_plus('endHh') : '23',               # 끝 시간 설정\n",
    "                        quote_plus('stnIds') : f'{std}'})         # 지점번호 선택\n",
    "\n",
    "                # request 객체\n",
    "                request = urllib.request.Request(url + unquote(queryParams))\n",
    "\n",
    "                # url 응답 객체 저장\n",
    "                response_body = urlopen(request, timeout=60).read()\n",
    "\n",
    "                # 디코딩\n",
    "                decode_data = response_body.decode('utf-8')\n",
    "\n",
    "                # string인 xml 파싱\n",
    "                xml_parse = xmltodict.parse(decode_data)     \n",
    "                xml_dict = json.loads(json.dumps(xml_parse))\n",
    "                result = xml_dict['response']['header']['resultMsg']\n",
    "\n",
    "                # 무한 루프를 돌리기 때문에 try-exept 를 빠져나오지 못하는 'NO_DATA'오류일 경우 반복문을 빠져나오게 함\n",
    "                if result == 'NO_DATA':\n",
    "                    breaker = True\n",
    "                    break\n",
    "                dicts = xml_dict['response']['body']['items']['item']\n",
    "                \n",
    "                # 데이터 프레임 컬럼 설정\n",
    "                df_temp = pd.DataFrame(columns = ['날짜', '시간', '지점_번호', '지점명', '기온', '강수량', '풍속', '풍향',\n",
    "                                                  '습도', '증기압', '이슬점온도', '현지기압', '해면기압', '일조', '일사', '적설',\n",
    "                                                  '3시간_신절설', '전운량', '중하층운량', '운형', '최저운고', '시정', '지면온도',\n",
    "                                                  '5cm_지중온도', '10cm_지중온도', '20cm_지중온도', '30cm_지중온도'])\n",
    "                \n",
    "                # 받은 데이터 데이터 프레임에 적재\n",
    "                for i in range(len(dicts)):\n",
    "                    df_temp.loc[i] = [dicts[i]['tm'][:10], dicts[i]['tm'][11:], dicts[i]['stnId'], dicts[i]['stnNm'],\n",
    "                                      dicts[i]['ta'], dicts[i]['rn'], dicts[i]['ws'], dicts[i]['wd'], dicts[i]['hm'],\n",
    "                                      dicts[i]['pv'], dicts[i]['td'], dicts[i]['pa'], dicts[i]['ps'], dicts[i]['ss'],\n",
    "                                      dicts[i]['icsr'], dicts[i]['dsnw'], dicts[i]['hr3Fhsc'], dicts[i]['dc10Tca'],\n",
    "                                      dicts[i]['dc10LmcsCa'], dicts[i]['clfmAbbrCd'], dicts[i]['lcsCh'], dicts[i]['vs'],\n",
    "                                      dicts[i]['ts'], dicts[i]['m005Te'], dicts[i]['m01Te'], dicts[i]['m02Te'], dicts[i]['m03Te']]\n",
    "                df = pd.concat([df, df_temp])\n",
    "                print(f'{page}page 완료', end = ', ')\n",
    "                page += 1\n",
    "\n",
    "            except:\n",
    "                print(f'{page}page 오류 다시 시작', end = ', ')\n",
    "            if page == 11:\n",
    "                break\n",
    "        print()\n",
    "        print(f'{year}년 완료 -- {datetime.now().time()}')\n",
    "\n",
    "    # 지점 별로 csv 파일로 저장\n",
    "    if len(df.values) != 0:\n",
    "        df.to_csv(f\"./data/data_hour/test/{df['지점명'].unique()[0]}\" + \"_tm_final.csv\", index=False)\n",
    "        print(f'지점번호 {std} 완료 -- {datetime.now().time()}\\n')\n",
    "    else:\n",
    "        print(f'지점번호 {std} NO DATA\\n')"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "0add23f5",
   "metadata": {},
   "source": [
    "일별 테스트(test) 데이터의 결측치를 대체하기 위해서 2021 ~ 2022년의 시간별 데이터를 수집합니다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": 8,
   "id": "e256b9d5-4c76-4a21-82b6-3b604026fbd7",
   "metadata": {
    "jupyter": {
     "outputs_hidden": true,
     "source_hidden": true
    },
    "scrolled": true,
    "tags": []
   },
   "outputs": [],
   "source": [
    "url = 'http://apis.data.go.kr/1360000/AsosHourlyInfoService/getWthrDataList'\n",
    "\n",
    "std_list = [90, 93, 95, 98, 99, 100, 101, 102, 104, 105, 106, 108, 112, 114, 115, 119, 121, 127, 129, 130,\n",
    "            131, 133, 135, 136, 137, 138, 140, 143, 146, 152, 155, 156, 159, 162, 165, 168, 169, 170, 172,\n",
    "            174, 177, 184, 185, 188, 189, 192, 201, 202, 203, 211, 212, 216, 217, 221, 226, 232, 235, 236,\n",
    "            238, 239, 243, 244, 245, 247, 248, 251, 252, 253, 254, 255, 257, 258, 259, 260, 261, 262, 263,\n",
    "            264, 266, 268, 271, 272, 273, 276, 277, 278, 279, 281, 283, 284, 285, 288, 289, 294, 295]\n",
    "\n",
    "for std in std_list:\n",
    "    df = pd.DataFrame()\n",
    "    print(f'▷ 지점번호 {std} 시작 -- {datetime.now().time()}')\n",
    "    breaker = False\n",
    "    for year in range(21, 23, 1):\n",
    "        print(f'{year}년 시작')\n",
    "        if breaker == True:\n",
    "            break\n",
    "        page = 1\n",
    "        # breaker = False\n",
    "\n",
    "        while True:\n",
    "            try:\n",
    "                queryParams = '?' + urlencode({\n",
    "                        quote_plus('ServiceKey') : '%2FNJpEb%2B4mPrdl%2BjK0N5XfHVW2zDqJiBAj7IwHykCI4aJExuDhIr9VfQeUYZKCAeIDNMRk%2BfUF0x9J47VhrxPyQ%3D%3D',\n",
    "                        quote_plus('pageNo') : f'{page}',         # 1 ~ 10\n",
    "                        quote_plus('numOfRows') : '880',          # 윤년 때문에 366으로 설정\n",
    "                        quote_plus('dataType') : 'XML',\n",
    "                        quote_plus('dataCd') : 'ASOS',\n",
    "                        quote_plus('dateCd') : 'HR',\n",
    "                        quote_plus('startDt') : f'20{year}0101',\n",
    "                        quote_plus('endDt') : f'20{year}1231',\n",
    "                        quote_plus('startHh') : '00',\n",
    "                        quote_plus('endHh') : '23',\n",
    "                        quote_plus('stnIds') : f'{std}'})\n",
    "\n",
    "                request = urllib.request.Request(url + unquote(queryParams))\n",
    "\n",
    "                response_body = urlopen(request, timeout=60).read()\n",
    "\n",
    "                decode_data = response_body.decode('utf-8')\n",
    "\n",
    "                xml_parse = xmltodict.parse(decode_data)         # string인 xml 파싱\n",
    "                xml_dict = json.loads(json.dumps(xml_parse))\n",
    "                result = xml_dict['response']['header']['resultMsg']\n",
    "                if result == 'NO_DATA':\n",
    "                    breaker = True\n",
    "                    break\n",
    "\n",
    "                dicts = xml_dict['response']['body']['items']['item']\n",
    "\n",
    "                df_temp = pd.DataFrame(columns = ['날짜', '시간', '지점_번호', '지점명', '기온', '강수량', '풍속', '풍향',\n",
    "                                                  '습도', '증기압', '이슬점온도', '현지기압', '해면기압', '일조', '일사', '적설',\n",
    "                                                  '3시간_신절설', '전운량', '중하층운량', '운형', '최저운고', '시정', '지면온도',\n",
    "                                                  '5cm_지중온도', '10cm_지중온도', '20cm_지중온도', '30cm_지중온도'])\n",
    "\n",
    "                for i in range(len(dicts)):\n",
    "                    df_temp.loc[i] = [dicts[i]['tm'][:10], dicts[i]['tm'][11:], dicts[i]['stnId'], dicts[i]['stnNm'],\n",
    "                                      dicts[i]['ta'], dicts[i]['rn'], dicts[i]['ws'], dicts[i]['wd'], dicts[i]['hm'],\n",
    "                                      dicts[i]['pv'], dicts[i]['td'], dicts[i]['pa'], dicts[i]['ps'], dicts[i]['ss'],\n",
    "                                      dicts[i]['icsr'], dicts[i]['dsnw'], dicts[i]['hr3Fhsc'], dicts[i]['dc10Tca'],\n",
    "                                      dicts[i]['dc10LmcsCa'], dicts[i]['clfmAbbrCd'], dicts[i]['lcsCh'], dicts[i]['vs'],\n",
    "                                      dicts[i]['ts'], dicts[i]['m005Te'], dicts[i]['m01Te'], dicts[i]['m02Te'], dicts[i]['m03Te']]\n",
    "                df = pd.concat([df, df_temp])\n",
    "                print(f'{page}page 완료', end=', ')\n",
    "                page += 1\n",
    "\n",
    "            except:\n",
    "                print(f'{page}page 오류 다시 시작', end=', ')\n",
    "            if page == 3:\n",
    "                break\n",
    "        print()\n",
    "        print(f'{year}년 완료 -- {datetime.now().time()}')\n",
    "    \n",
    "    if len(df.values) != 0:\n",
    "        df.to_csv(f\"./data/data_hour/test/{df['지점명'].unique()[0]}\" + \"_tm_fianl.csv\", index=False)\n",
    "        print(f'지점번호 {std} 완료 -- {datetime.now().time()}\\n')\n",
    "    else:\n",
    "        print(f'지점번호 {std} NO DATA\\n')"
   ]
  },
  {
   "attachments": {
    "1.png": {
     "image/png": "..."
    }
   },
   "cell_type": "markdown",
   "id": "3347428e",
   "metadata": {},
   "source": [
    "![1.png](attachment:1.png)"
   ]
  },
  {
   "cell_type": "markdown",
   "id": "4fc3c8eb",
   "metadata": {},
   "source": [
    "행정안전부의 복구지원과에서 매년 재해연보로 재해 데이터를 정리하는데, 피해규모에 대한 데이터는 국민재난안전포털에서 수집했습니다."
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "37950c2d",
   "metadata": {},
   "outputs": [],
   "source": []
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3 (ipykernel)",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.12"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 5
}